## ML experimentation
#### Current approach:
- Selection of 3 different encoders (TokenVectorizer, 2x BERT embeddings)
- All encoders use CodeBERT as a tokenizer
- line, prev_line, next_line & func_vul are considered
- MLPClassifier, not optimized yet


#### Open issues:
- Try different ML models
- Include joern-features (maybe, but unlikely due to resource constraints)
- Hyperparameter tuning
- ...

_References_:
- https://www.analyticsvidhya.com/blog/2019/09/demystifying-bert-groundbreaking-nlp-framework/
- https://github.com/microsoft/CodeBERT

In [1]:
import pandas as pd
import numpy as np
import math
import torch
from tqdm import tqdm
import itertools
from line_encoders import EncoderCountVectorizer, EncoderTFIDFVectorizer, EncoderBERTVectorConcat, EncoderBERTStringConcat

In [2]:
# DO NOT forget 'keep_default_na=False' --> otherwise some NaN values in read data
df = pd.read_csv('./big-vul_dataset/line_sample_all_balanced_ratio.csv', skipinitialspace=True, low_memory=True, keep_default_na=False)

# DO NOT forget 'keep_default_na=False' --> otherwise some NaN values in read data
df_functions = pd.read_csv('./big-vul_dataset/functions_only_all.csv', usecols=['func_id', 'target'], skipinitialspace=True, low_memory=True, keep_default_na=False)
df = pd.merge(df, df_functions, on='func_id', how='inner')

In [3]:
df.head()

,Unnamed: 0,func_id,line,vul,prev_line,next_line,target
0,0,177738,} while (*ptr);,1,ptr++;,s->have_period = 1;,1
1,57863,177738,if ((YYLIMIT - YYCURSOR) < 3) YYFILL(3);,0,++YYCURSOR;,yych = *YYCURSOR;,1
2,69617,177738,"YYDEBUG(11, *YYCURSOR);",0,yy11:,yych = *++YYCURSOR;,1
3,1,177739,if (TLS1_get_version(s) >= TLS1_2_VERSION &&,1,long alg2 = s->s3->tmp.new_cipher->algorithm2;,alg2 == (SSL_HANDSHAKE_MAC_DEFAULT|TLS1_PRF)),1
4,2,177740,if (timestr->length < 13) {,1,long gmadjust = 0;,"php_error_docref(NULL TSRMLS_CC, E_WARNING, ""e...",1


## Data preparation section

- Selection of 3 different data encoders
- Create train and test dataset
- Split into batches

### Please select _encoder_ below!
- `encoder = EncoderCountVectorizer(df_data=df, vocabulary_path="vec_vocabulary.pkl")`
- `encoder = EncoderBERTVectorConcat()`
- `encoder = EncoderBERTStringConcat()`

In [4]:
### Use a standard vectorization approach by creating a feature matrix.
### Loses semantic and structural information, but it is FAST (>> anything BERT embedding related)
### Pre-configured to use 750 features 
encoder = EncoderCountVectorizer(df_data=df, vocabulary_path="vocab/vec_vocabulary.pkl")
#encoder = EncoderTFIDFVectorizer(df_data=df)
#encoder = EncoderCountVectorizer(df_data=df)

### Most precise since the embeddings of all 3 features are preserved (i.e. generates a 2304-element vector)
### On average 2x slower than EncoderBERTStringConcat
### Set avg_embeddings=False to take the first embedding vector, or True to average all embedding vectors

#encoder = EncoderBERTVectorConcat(avg_embeddings=False)

### Less precise since all 3 features are concatenated before embedding creation (i.e. generates a 768-element vector)
### On average 2x faster than EncoderBERTVectorConcat
### Set avg_embeddings=False to take the first embedding vector, or True to average all embedding vectors

#encoder = EncoderBERTStringConcat()

D:\Elias\Documents\AAU\Automating_SE\automating_se\line_encoders.py:29: UserWarning: Both df_data and vocabulary_path are set. Ignoring df_data.
  warnings.warn("Both df_data and vocabulary_path are set. Ignoring df_data.")


In [ ]:
test = encoder.encode(["int x = 5", "int x = 5"], ["function(){", "function(){"], ["}", "}"], [0,1])
test

In [5]:
from sklearn.model_selection import train_test_split

# split into training and validation sets
line_tr, line_test, prev_line_tr, prev_line_test, next_line_tr, next_line_test, func_vul_tr, func_vul_test, y_tr, y_test = \
    train_test_split(df['line'], df['prev_line'], df['next_line'], df['target'], df['vul'], test_size=0.2, random_state=42)

In [6]:
# define how big a batch of entries should be (depending on RAM)
batch_size = 100

# number of epochs is calulated based on the batch_size
epochs = math.ceil(len(line_tr)/batch_size)

# split the dataframes (X_tr, y_tr) into an array of dataframes (number of epochs)
batchesLine = np.array_split(line_tr, epochs)
batchesPrevLine = np.array_split(prev_line_tr, epochs)
batchesNextLine = np.array_split(next_line_tr, epochs)
batchesFuncVul = np.array_split(func_vul_tr, epochs)
batchesY = np.array_split(y_tr, epochs)

## ML Model

- Using train_test_split for creating subsets of data
- Training MLPClassifier with training data
- Evaluating model with test data
- (Include RandomForestClassifier as one alternative option for model)


In [7]:
# import and initialisation of generic MLPClassifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation='relu', alpha=0.05, hidden_layer_sizes=(1500,750), learning_rate='adaptive',solver='adam', shuffle=True)

# iterate over the number of epochs
for i in tqdm(range(epochs)):
    # take a batch and process it and partial_fit the model to the batch
    line_batch, prev_line_batch, next_line_batch, func_vul, Y_batch = \
        batchesLine[i], batchesPrevLine[i], batchesNextLine[i], batchesFuncVul[i], batchesY[i]
    
    # encode to vector 
    encodedBatch = encoder.encode(line_batch.tolist(), prev_line_batch.tolist(), \
                                  next_line_batch.tolist(), func_vul.tolist())
    
    clf.partial_fit(encodedBatch, Y_batch, classes=np.unique(y_tr))

100%|████████████████████| 600/600 [12:38<00:00,  1.26s/it]


In [ ]:
# RandomForestClassifier (Currently WIP)
# Does not provide a partial_fit() method, therefore workaround by increasing tree count each epoch

TREE_INCREASE_EACH_EPOCH = 10

# import and initialisation of generic MLPClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, warm_start=True)

# iterate over the number of epochs
for i in tqdm(range(epochs)):
    # take a batch and process it and partial_fit the model to the batch
    line_batch, prev_line_batch, next_line_batch, func_vul, Y_batch = \
        batchesLine[i], batchesPrevLine[i], batchesNextLine[i], batchesFuncVul[i], batchesY[i]
    
    # encode to vector 
    encodedBatch = encoder.encode(line_batch.tolist(), prev_line_batch.tolist(), \
                                  next_line_batch.tolist(), func_vul.tolist())
    
    clf.fit(encodedBatch, Y_batch)
    # increase tree count each epoch
    clf.set_params(n_estimators=len(clf.estimators_)+TREE_INCREASE_EACH_EPOCH)

In [ ]:
# Naive Bayes approach

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

# iterate over the number of epochs
for i in tqdm(range(epochs)):
    # take a batch and process it and partial_fit the model to the batch
    line_batch, prev_line_batch, next_line_batch, func_vul, Y_batch = \
        batchesLine[i], batchesPrevLine[i], batchesNextLine[i], batchesFuncVul[i], batchesY[i]
    
    # encode to vector 
    encodedBatch = encoder.encode(line_batch.tolist(), prev_line_batch.tolist(), \
                                  next_line_batch.tolist(), func_vul.tolist())
    
    clf.partial_fit(encodedBatch, Y_batch, classes=np.unique(y_tr))



In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()

# iterate over the number of epochs
for i in tqdm(range(epochs)):
    # take a batch and process it and partial_fit the model to the batch
    line_batch, prev_line_batch, next_line_batch, func_vul, Y_batch = \
        batchesLine[i], batchesPrevLine[i], batchesNextLine[i], batchesFuncVul[i], batchesY[i]
    
    # encode to vector 
    encodedBatch = encoder.encode(line_batch.tolist(), prev_line_batch.tolist(), \
                                  next_line_batch.tolist(), func_vul.tolist())
    
    clf.partial_fit(encodedBatch, Y_batch, classes=np.unique(y_tr))



In [ ]:
X_test_encoded = encoder.encode(line_test.tolist(), prev_line_test.tolist(), \
                                next_line_test.tolist(), func_vul_test.tolist())

print("Accuracy of prediction: " , clf.score(X_test_encoded, y_test))
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test_encoded)
print(classification_report(y_test, y_pred))

## Model persistance
Selectively execute when needed!

In [8]:
from joblib import dump
# Store model
dump(clf, 'models/mlp_Count_withFuncVul.model')

['models/mlp_Count_withFuncVul.model']

In [ ]:
from joblib import load
# Load model
clf = load('models/mlp_Count_withFuncVul.model') 

## Hyperparameter tuning

<span style="color:red">Currently not _really_ supported due to missing batch support. <br>i.e. only works for small data samples and _EncoderCountVectorizer_</span> 

In [ ]:
mlp = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

# TODO: add batch support to allow for larger data end even alternative encoders

encoded_X_tr = encoder.encode(line_tr.tolist(), prev_line_tr.tolist(), next_line_tr.tolist())

from sklearn.model_selection import GridSearchCV

gridclf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
gridclf.fit(encoded_X_tr, y_tr)


# Best parameter set
print('Best parameters found:\n', gridclf.best_params_)

# All results
means = gridclf.cv_results_['mean_test_score']
stds = gridclf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gridclf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rndclf = RandomForestClassifier(max_depth=2, random_state=0)
rndclf.fit(X_train, y_train)
rndclf.score(X_test, y_test)

y_pred = rndclf.predict(X_test)
print(classification_report(y_test, y_pred))



## Validation section

- Read unbalanced dataset
- Encode using selected encoder
- Evaluate model

In [9]:
# Load data for validation of the whole process

# DO NOT forget 'keep_default_na=False' --> otherwise some NaN values in read data
val_df = pd.read_csv('./big-vul_dataset/line_sample_1p_original_ratio.csv', skipinitialspace=True, low_memory=True, keep_default_na=False)
df_functions = pd.read_csv('./big-vul_dataset/functions_only_all.csv', skipinitialspace=True, low_memory=True, keep_default_na=False)
val_df = pd.merge(val_df, df_functions, on='func_id', how='inner')
val_df = val_df.reset_index()

In [10]:
from joblib import load
# Load function model
clf_func = load('models/full_function.model')

from function_encoders import FuncEncoderCountVectorizer
encoder_func = FuncEncoderCountVectorizer(vocabulary_path="vocab/func_vocab.pkl")

line_val, prev_line_val, next_line_val, func_vul, func_code, y_val = \
    val_df['line'], val_df['prev_line'], val_df['next_line'], val_df['target'], val_df['processed_func'], val_df['vul']

from sklearn.metrics import classification_report
func_vul_pred = clf_func.predict(encoder_func.encode(func_code.tolist()))
print(classification_report(func_vul, func_vul_pred))

Token indices sequence length is longer than the specified maximum sequence length for this model (732 > 512). Running this sequence through the model will result in indexing errors


              precision    recall  f1-score   support

           0       0.94      1.00      0.97     39830
           1       0.99      0.66      0.79      7354

    accuracy                           0.95     47184
   macro avg       0.97      0.83      0.88     47184
weighted avg       0.95      0.95      0.94     47184



In [11]:
X_val_encoded = encoder.encode(line_val.tolist(), prev_line_val.tolist(), \
                               next_line_val.tolist(), func_vul_pred.tolist())

print("Accuracy of prediction: " , clf.score(X_val_encoded, y_val))
from sklearn.metrics import classification_report
y_val_pred = clf.predict(X_val_encoded)
print(classification_report(y_val, y_val_pred))

Accuracy of prediction:  0.9093760596812479
              precision    recall  f1-score   support

           0       1.00      0.91      0.95     46809
           1       0.04      0.47      0.08       375

    accuracy                           0.91     47184
   macro avg       0.52      0.69      0.51     47184
weighted avg       0.99      0.91      0.95     47184



In [12]:
for input, prediction, label in zip(line_val, y_val_pred, y_val):
  if prediction != label:
    print(input, 'has been classified as ', prediction, 'and should be ', label)

g_array_append_val (ints, v_UINT); has been classified as  0 and should be  1
days_since_last_update); has been classified as  0 and should be  1
assert(m_entries == NULL); has been classified as  0 and should be  1
break; has been classified as  0 and should be  1
const char* event_name) { has been classified as  0 and should be  1
struct libusb_context *ctx = DEVICE_CTX(transfer->dev_handle->dev); has been classified as  0 and should be  1
int r = LIBUSB_ERROR_INVALID_PARAM; has been classified as  0 and should be  1
return LIBUSB_ERROR_NOT_FOUND; has been classified as  0 and should be  1
item->type = cJSON_Object; has been classified as  0 and should be  1
newitem->prev = c->prev; has been classified as  0 and should be  1
GL_ARRAY_BUFFER, 0, num_vertices * sizeof(GLfloat) * 4, _)) has been classified as  0 and should be  1
EXPECT_CALL(*gl_, BufferSubData( has been classified as  0 and should be  1
ctxt->dst.type = OP_REG; has been classified as  0 and should be  1
/* ex3: to ee_bl

UINT8            old_sec_state; has been classified as  1 and should be  0
hide_shell(shell_visible ? 1 : 0); has been classified as  1 and should be  0
} else if (is_connected) { has been classified as  1 and should be  0
if (evt->key.flags & GF_KEY_MOD_CTRL) has been classified as  1 and should be  0
} else if (memcmp(CharBuf, "MM", 2) == 0) { has been classified as  1 and should be  0
|| ImageInfo->Thumbnail.size <= 0 has been classified as  1 and should be  0
case TAG_FMT_DOUBLE: has been classified as  1 and should be  0
|| ImageInfo->Thumbnail.offset <= 0 has been classified as  1 and should be  0
#define FOUND_THUMBNAIL     (1<<SECTION_THUMBNAIL) has been classified as  1 and should be  0
info_data->tag = TAG_FMT_UNDEFINED; has been classified as  1 and should be  0
xp_field->size = exif_process_string_raw(&xp_field->value, szValuePtr, ByteCount); has been classified as  1 and should be  0
s_den = php_ifd_get32s(4+(char *)value, motorola_intel); has been classified as  1 and sho

break; has been classified as  1 and should be  0
(void) WriteBlobString(image,buffer); has been classified as  1 and should be  0
else has been classified as  1 and should be  0
iput(inode); has been classified as  1 and should be  0
sm_info->rec_prefree_segments = DEF_MAX_RECLAIM_PREFREE_SEGMENTS; has been classified as  1 and should be  0
quantum_type=IndexAlphaQuantum; has been classified as  1 and should be  0
DecodeLabImage(image,&image->exception); has been classified as  1 and should be  0
if (image_info->sampling_factor != (char *) NULL) has been classified as  1 and should be  0
(void) TIFFSetField(tiff,TIFFTAG_IMAGEWIDTH,(uint32) image->columns); has been classified as  1 and should be  0
break; has been classified as  1 and should be  0
if (TIFFWritePixels(tiff,&tiff_info,y,2,image) == -1) has been classified as  1 and should be  0
else has been classified as  1 and should be  0
if (specifics.has_browser_type()) { has been classified as  1 and should be  0
for (int i = 0; i

break; has been classified as  1 and should be  0
dom_object *intern; has been classified as  1 and should be  0
resolve_externals = doc_props->resolveexternals; has been classified as  1 and should be  0
else has been classified as  1 and should be  0
NULL, NULL); has been classified as  1 and should be  0
int				    numScreens; has been classified as  1 and should be  0
register const z_crc_t FAR *buf4; has been classified as  1 and should be  0
tables[i]     = tables[i + 1]; has been classified as  1 and should be  0
if (KEY_TYPE_AES == exdata->smtype) has been classified as  1 and should be  0
ctx->idsp.idct_put(dest_v,                               dct_linesize_chroma, row->blocks[4]); has been classified as  1 and should be  0
if (!ctx->is_444) { has been classified as  1 and should be  0
} else if (is_write) { has been classified as  1 and should be  0
if (ret < 0 has been classified as  1 and should be  0
return GNUTLS_E_DECOMPRESSION_FAILED; has been classified as  1 and shoul

(!timing.parse_stop || timing.parse_stop > first_background_time)) { has been classified as  1 and should be  0
PAGE_LOAD_HISTOGRAM( has been classified as  1 and should be  0
if (WasStartedInForegroundOptionalEventInForeground(timing.first_paint, has been classified as  1 and should be  0
} has been classified as  1 and should be  0
here->e_value_size = cpu_to_le32(value_len); has been classified as  1 and should be  0
break; has been classified as  1 and should be  0
size_t has been classified as  1 and should be  0
ThrowReaderException(CorruptImageError, has been classified as  1 and should be  0
if (c != EOF) has been classified as  1 and should be  0
if (matte < 0) has been classified as  1 and should be  0
int error; has been classified as  1 and should be  0
} else { has been classified as  1 and should be  0
endp = strings + strings_size; has been classified as  1 and should be  0
if (avio_feof(pb)) has been classified as  1 and should be  0
nsv->index_entries = table_entries_u

...

return rtnVal; has been classified as  1 and should be  0
return handled; has been classified as  1 and should be  0
if (usb_endpoint_xfer_isoc(dep->endpoint.desc)) has been classified as  1 and should be  0
ent2->type == ST_STRUCT && Z_TYPE(ent2->data) == IS_ARRAY) { has been classified as  1 and should be  0
clone = skb_clone(skb, GFP_ATOMIC); has been classified as  1 and should be  0
cstm(J, F, stm->d); has been classified as  1 and should be  0
case STM_DEBUGGER: has been classified as  1 and should be  0
ctryfinally(J, F, stm->a, stm->d); has been classified as  1 and should be  0
} has been classified as  1 and should be  0
if (IsFloatDefined(cin.image.green_primary_chromaticity[1]) != MagickFalse) has been classified as  1 and should be  0
switch (cin.image.orientation) has been classified as  1 and should be  0
ThrowReaderException(CorruptImageError,"ImproperImageHeader"); has been classified as  1 and should be  0
kfree(uref_multi); has been classified as  1 and should be  0


goto error; has been classified as  1 and should be  0
return err; has been classified as  1 and should be  0
USB_RECIP_DEVICE, 0, 0, data, 8, has been classified as  1 and should be  0
RFSFile *file; has been classified as  1 and should be  0
r_cons_printf ("%10s  %s\n", plug->name, plug->desc); has been classified as  1 and should be  0
} else eprintf ("Cannot open file\n"); has been classified as  1 and should be  0
(IsStringTrue(option) == MagickFalse))) has been classified as  1 and should be  0
.init_ctx = init_ctx, has been classified as  1 and should be  0
tcp_parse_options(skb, &tcp_opt, 0, NULL); has been classified as  1 and should be  0
static void __svc_rdma_free(struct work_struct *work) has been classified as  1 and should be  0
return true; has been classified as  1 and should be  0
return WebBindings::retainObject(window_npobject_); has been classified as  1 and should be  0
p++; has been classified as  1 and should be  0
sizeof(*sun_colormap)); has been classified as 

} has been classified as  1 and should be  0
kCrosHalV3BufferManagerFailedToRegisterBuffer; has been classified as  1 and should be  0
kWinDirectShowFailedToConnectTheCaptureGraph; has been classified as  1 and should be  0
case media::VideoCaptureError::kV4L2ErrorRequestingMmapBuffers: has been classified as  1 and should be  0
return media::mojom::VideoCaptureError::kV4L2ErrorRequestingMmapBuffers; has been classified as  1 and should be  0
case media::VideoCaptureError::kV4L2FailedToDequeueCaptureBuffer: has been classified as  1 and should be  0
http_response_send(res); has been classified as  1 and should be  0
HB_UShort          cl1 = 0, cl2 = 0; has been classified as  1 and should be  0
for (line = h - 1; line >= 0 ; line--) { has been classified as  1 and should be  0
krb5_principal              cprinc=NULL; has been classified as  1 and should be  0
rlcinf->deciphered[0] = FALSE; has been classified as  1 and should be  0
if ((dls.artCount == 0) || (dls.artCount > DLS_MAX_ART

child_set_env(&env, &envsize, s->env[i].name, has been classified as  1 and should be  0
p += 9; has been classified as  1 and should be  0
if (mCodecContext == NULL) { has been classified as  1 and should be  0
(uint64_t)cfg->g_timebase.num / has been classified as  1 and should be  0
return -EFAULT; has been classified as  1 and should be  0
static void conn_free(struct connectdata *conn) has been classified as  1 and should be  0
size_t				rep_len	= 0; has been classified as  1 and should be  0
goto encode_amp; has been classified as  1 and should be  0
AddGenericDllEvictionPolicy(policy); has been classified as  1 and should be  0
if (opt.hsts && hsts_store) has been classified as  1 and should be  0
bw.i = EXTRACT_32BITS(tptr); has been classified as  1 and should be  0
return v8::Undefined(GetIsolate()); has been classified as  1 and should be  0
net = container_of(qp->q.net, struct net, ipv4.frags); has been classified as  1 and should be  0
umts_fp_conversation_info->crnc_port 

{	psf_log_printf (psf, "Bad item offset (%d).\n", rsrc.item_offset) ; has been classified as  1 and should be  0
SIGTRAP) == NOTIFY_STOP) has been classified as  1 and should be  0
return -1L; has been classified as  1 and should be  0
int ecpointformatlist_length = *(sdata++); has been classified as  1 and should be  0
content::BrowsingDataRemover::ORIGIN_TYPE_UNPROTECTED_WEB, has been classified as  1 and should be  0
u_int offset; has been classified as  1 and should be  0
return image; has been classified as  1 and should be  0
pin_info.max_tries = 3; has been classified as  1 and should be  0
av_log(s, AV_LOG_DEBUG, "fileversion          = %d\n", ape_ctx->fileversion); has been classified as  1 and should be  0
base::IntToString(tab_id_)); has been classified as  1 and should be  0
sem_lock_and_putref(sma); has been classified as  1 and should be  0
List<BufferInfo *> &inQueue = getPortQueue(kInputPortIndex); has been classified as  1 and should be  0
const char *proxyauth, int *n